In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [3]:
# Basics
import tensorflow as tf
import xarray as xr

# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *  # CRPS metrics
from src.utils.data_split import *  # Splitting data into X and y
from src.utils.drn_make_X_array import *  # Import make train array functions (make_X_array)
from src.models.EMOS import *  # EMOS implementation
from src.models.DRN.DRN_model import * # DRN implementation
from src.models.EMOS_global.EMOS_global_load_score import *  # Load EMOS_global_scores
from src.models.EMOS_global.EMOS_global_load_model import *  # Load EMOS_global_models
import data.raw.load_data_raw as ldr  # Load raw data
import data.processed.load_data_processed as ldp  # Load processed data normed
import data.processed.load_data_processed_denormed as ldpd  # Load processed data denormed
from src.models.CRPS_baseline.CRPS_load import *  # Load CRPS scores

<IPython.core.display.Javascript object>

### 0. Set up

In [4]:
lead_time = 0

<IPython.core.display.Javascript object>

### 1. Load Data

In [5]:
dat_train_denorm = ldpd.load_data_all_train_proc_denorm()
dat_test_denorm = ldpd.load_data_all_test_proc_denorm()

<IPython.core.display.Javascript object>

### 2. Data Split

In [25]:
dat_X_train_lead_all_denorm, dat_y_train_lead_all_denorm = split_var_lead(
    dat_train_denorm
)
dat_X_test_lead_all_denorm, dat_y_test_lead_all_denorm = split_var_lead(dat_test_denorm)

<IPython.core.display.Javascript object>

### 3. Data preparation

Idea: X_train constructed by flattening the value arrays and concatenating them together, so that in the end I get the shape (n, n_variables), keep in mind we use both mean and std of each variabel for training. So in total we will have 10 / 12, depending on which variable. Important, for drn we'll also need the embeddings, hence the idea to keep another flattened array with shape (n, 1) with the complementing embedding (1 - 15600).

In [33]:
drn_X_train_0_array, drn_embedding_train_0_array = make_X_array(
    dat_X_train_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [ ]:
path = "/Data/Delong_BA_Data/mean_std_max_values/denorm/"

In [63]:
def make_mean_std_max(dat_var_lead_all):
    path = "/Data/Delong_BA_Data/mean_std_max_values/denorm/"
    mean_max = np.zeros([6, 31])
    std_max = np.zeros([6, 31])
    for var in range(6):
        for lead_time in range(31):
            mean_max[var, lead_time] = (
                dat_var_lead_all[var][lead_time].isel(mean_std=0).max().values
            )
            std_max[var, lead_time] = (
                dat_var_lead_all[var][lead_time].isel(mean_std=1).max().values
            )
    np.save("/Data/Delong_BA_Data/mean_std_max_values/denorm/mean_max.npy", mean_max)
    np.save("/Data/Delong_BA_Data/mean_std_max_values/denorm/std_max.npy", std_max)

<IPython.core.display.Javascript object>

In [42]:
make_mean_std_max(dat_X_train_lead_all_denorm)

31

<IPython.core.display.Javascript object>

In [58]:
test = dat_X_train_lead_all_denorm[0][0].isel(mean_std=0).max().values

<IPython.core.display.Javascript object>

In [62]:
np.zeros([5, 31])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

<IPython.core.display.Javascript object>

In [34]:
drn_X_test_0_array, drn_embedding_test_0_array = make_X_array(
    dat_X_test_lead_all_denorm, 0
)

<IPython.core.display.Javascript object>

In [35]:
t2m_y_train = dat_y_train_lead_all_denorm[2][0].values.flatten()
t2m_y_test = dat_y_test_lead_all_denorm[2][0].values.flatten()

<IPython.core.display.Javascript object>

### 4. Model SetUp

In [36]:
hidden_layer = []
emb_size = 3
max_id = 15600 - 1

<IPython.core.display.Javascript object>

In [37]:
drn_0_model = build_emb_model(12, 2, hidden_layer, emb_size, max_id, compile=True)

/home/dchen/.conda/envs/Bach_Del/lib/python3.9/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


<IPython.core.display.Javascript object>

In [39]:
drn_0_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 embedding_3 (Embedding)        (None, 1, 3)         46800       ['input_8[0][0]']                
                                                                                                  
 input_7 (InputLayer)           [(None, 12)]         0           []                               
                                                                                                  
 flatten_3 (Flatten)            (None, 3)            0           ['embedding_3[0][0]']            
                                                                                            

<IPython.core.display.Javascript object>

### 5. Train DRN for lead_time 0, t2m

In [40]:
drn_0_model.fit(
    [drn_X_train_0_array, drn_embedding_train_0_array],
    t2m_y,
    epochs=10,
    batch_size=4096,
    validation_data=[[drn_X_test_0_array, drn_embedding_test_0_array], t2m_y_test],
)

2023-05-31 15:40:31.778668: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 1070035200 exceeds 10% of free system memory.


Epoch 1/10
   1/5443 [..............................] - ETA: 2:04:55 - loss: 13649.6836

InvalidArgumentError:  indices[1234,0] = 15600 is not in [0, 15600)
	 [[node model_3/embedding_3/embedding_lookup
 (defined at /home/dchen/.conda/envs/Bach_Del/lib/python3.9/site-packages/keras/layers/embeddings.py:191)
]] [Op:__inference_train_function_1175]

Errors may have originated from an input operation.
Input Source operations connected to node model_3/embedding_3/embedding_lookup:
In[0] model_3/embedding_3/embedding_lookup/800:	
In[1] model_3/embedding_3/Cast (defined at /home/dchen/.conda/envs/Bach_Del/lib/python3.9/site-packages/keras/layers/embeddings.py:190)

Operation defined at: (most recent call last)
>>>   File "/home/dchen/.conda/envs/Bach_Del/lib/python3.9/runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, None,
>>> 
>>>   File "/home/dchen/.conda/envs/Bach_Del/lib/python3.9/runpy.py", line 87, in _run_code
>>>     exec(code, run_globals)
>>> 
>>>   File "/home/dchen/.conda/envs/Bach_Del/lib/python3.9/site-packages/ipykernel_laun

<IPython.core.display.Javascript object>